# Futures trend following with custom args

In [1]:
# Import libraries
import xarray as xr
import xarray.ufuncs as xruf
import numpy as np
import pandas as pd

import qnt.output as qnout
import qnt.ta as qnta
import qnt.data    as qndata
import qnt.stepper as qnstepper
import qnt.stats   as qnstats
import qnt.graph   as qngraph
import datetime    as dt
import plotly.graph_objs as go
import xarray.ufuncs as xruf
import time

#### Load futures data
Quantnet provides data for 75 global derivatives. 
The underlying assets are currencies, cross-rates, indices, bonds, energy and metals from the world's futures exchanges.

Suppose we want to download the data for the last 4 years. One can use the following function:

In [2]:
fut_data = qndata.futures.load_data(tail = dt.timedelta(days = 16*365),
                        dims = ("time", "field", "asset"),
                        forward_order = True)
# The complete list
fut_data.asset

<xarray.DataArray 'asset' (asset: 75)>
array(['A6', 'AE', 'AH', 'B6', 'CB', 'CC', 'CF', 'CL', 'CT', 'D6', 'DM', 'DX',
       'DY', 'E6', 'ES', 'EW', 'F', 'FB', 'FP', 'FS', 'FY', 'G', 'GC', 'GE',
       'GF', 'GG', 'GX', 'HE', 'HF', 'HG', 'HR', 'J6', 'KC', 'L', 'L6', 'LE',
       'LF', 'LO', 'LS', 'M6', 'MX', 'N6', 'NG', 'NQ', 'NY', 'OJ', 'PA', 'PL',
       'QR', 'R6', 'RB', 'RF', 'RP', 'RY', 'S6', 'SB', 'SI', 'SZ', 'T6', 'TV',
       'VI', 'X', 'YM', 'ZB', 'ZC', 'ZF', 'ZL', 'ZM', 'ZN', 'ZO', 'ZQ', 'ZR',
       'ZS', 'ZT', 'ZW'], dtype=object)
Coordinates:
  * asset    (asset) object 'A6' 'AE' 'AH' 'B6' 'CB' ... 'ZR' 'ZS' 'ZT' 'ZW'

In [3]:
# we can see historical data on a chart
trend_fig = [
    go.Scatter(
        x = fut_data.sel(asset = 'B6').sel(field = 'close').to_pandas().index,
        y = fut_data.sel(asset = 'B6').sel(field = 'close'),
        line = dict(width=1,color='black'))]

# draw chart
fig = go.Figure(data = trend_fig)
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

## Weights allocation

This function calculates positions using wma and roc as trend indicators.

In [4]:
def calc_positions(futures, ma_periods, roc_periods, sideways_threshold):
    """ Calculates positions for given data(futures) and parameters """
    close = futures.sel(field='close')
    
    # calculate MA 
    ma = qnta.lwma(close, ma_periods)
    # calcuate ROC
    roc = qnta.roc(ma, roc_periods)

    # positive trend direction
    positive_trend = roc > sideways_threshold
    # negtive trend direction
    negative_trend = roc < -sideways_threshold 
    # sideways
    sideways_trend = abs(roc) <= sideways_threshold
    
    # We suppose that a sideways trend after a positive trend is also positive
    side_positive_trend = positive_trend.where(sideways_trend == False).ffill('time').fillna(False)
    # and a sideways trend after a negative trend is also negative
    side_negative_trend = negative_trend.where(sideways_trend == False).ffill('time').fillna(False)

    # define signals
    buy_signal = positive_trend
    buy_stop_signal = side_negative_trend

    sell_signal = negative_trend
    sell_stop_signal = side_positive_trend

    # calc positions 
    position = close.copy(True)
    position[:] = np.nan
    position = xr.where(buy_signal, 1, position)
    position = xr.where(sell_signal, -1, position)
    position = xr.where(xruf.logical_and(buy_stop_signal, position.ffill('time') > 0), 0, position)
    position = xr.where(xruf.logical_and(sell_stop_signal, position.ffill('time') < 0), 0, position)
    position = position.ffill('time').fillna(0)

    return position

Select asset and adjust parameters:

```python

asset = 'NY'

sdat = fut_data.sel(asset=asset).dropna('time','any')
sout = calc_positions(sdat,200, 5, 2)
sout = xr.concat([sout], pd.Index([asset], name='asset'))

ssta = qnstats.calc_stat(fut_data, sout)

display(ssta.to_pandas().tail())

performance = ssta.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

```

# This function calculate positions for multiple instruments with different parameters.

In [5]:
def calc_output_all(data, params):
    positions = data.sel(field='close').copy(True)
    positions[:] = np.nan
    for futures_name in params.keys(): 
        p = params[futures_name]
        futures_data = data.sel(asset=futures_name).dropna('time','any')
        p = calc_positions(futures_data, p['ma_periods'], p['roc_periods'], p['sideways_threshold'])
        positions.loc[{'asset':futures_name, 'time':p.time}] = p
    
    return positions

In [6]:
# say we select futures and their parameters for technical algorithm
params = {
    'NY': {
        'ma_periods': 200, 
        'roc_periods': 5, 
        'sideways_threshold': 2,
    },
    'ZO': {
        'ma_periods': 160, 
        'roc_periods': 10, 
        'sideways_threshold': 2,
    },
    'GX': {
        'ma_periods': 200, 
        'roc_periods': 20, 
        'sideways_threshold': 2
    },
}

futures_list = list(params.keys())

#form the output
output = calc_output_all(fut_data.sel(asset = futures_list), params)

## Statistics

In [7]:
stat = qnstats.calc_stat(fut_data, output)
stat.to_pandas().tail()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-12-10,2.670316,0.004794,0.174708,0.000000,-0.430911,0.322486,0.056341,1.0,3.0,0.069259,25.600232
2020-12-11,2.692470,0.008297,0.174720,0.000000,-0.430911,0.326803,0.057099,1.0,3.0,0.069258,25.600232
2020-12-14,2.690753,-0.000638,0.174716,-0.000638,-0.430911,0.324516,0.056698,1.0,3.0,0.069257,25.600232
2020-12-15,2.683238,-0.002793,0.174717,-0.003429,-0.430911,0.322111,0.056278,1.0,3.0,0.069256,25.600232
2020-12-16,2.680690,-0.000950,0.174632,-0.004375,-0.430911,0.313085,0.054675,1.0,3.0,0.069252,25.566467


In [8]:
# show plot with profit and losses:
performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

# Check result

In [9]:
qnout.check(output, fut_data)

Check missed dates...
Ok.
Check sharpe ratio.


ERROR! The sharpe ratio is too low. 0.31830742190747424 < 1


Check correlation.



WARNING! This strategy correlates with other strategies.


The number of systems with a larger Sharpe ratio and correlation larger than 0.8: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9999865317232406
Current sharpe ratio(5y): 1.0360038791167776



# Write output

In [10]:
qnout.write(output)

Write output: /root/fractions.nc.gz
